# 03 — Feature Engineering: Core Transaction Features

## Objective
This notebook focuses on engineering baseline fraud detection features using only
core transaction-level information that is always available at decision time.

Identity-based and time-derived features are intentionally excluded in this phase
to establish a robust and interpretable baseline.


In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

DATA_DIR = Path("..") / "data" / "raw"

train_tx = pd.read_csv(DATA_DIR / "train_transaction.csv")
train_id = pd.read_csv(DATA_DIR / "train_identity.csv")

train = train_tx.merge(
    train_id,
    on="TransactionID",
    how="left"
)

print(train.shape)


(590540, 434)


In [2]:
# Exclude obvious non-features
excluded_cols = {
    "TransactionID",
    "isFraud",
    "TransactionDT"
}

core_candidates = [
    c for c in train_tx.columns
    if c not in excluded_cols
]

len(core_candidates)


391

In [3]:
core_df = train_tx[core_candidates]

numeric_cols = core_df.select_dtypes(include=["int64", "float64"]).columns
categorical_cols = core_df.select_dtypes(include=["object"]).columns

print("Numeric transaction features:", len(numeric_cols))
print("Categorical-like transaction features:", len(categorical_cols))


Numeric transaction features: 377
Categorical-like transaction features: 14


C:\Users\Mambo\AppData\Local\Temp\ipykernel_21260\1141206156.py:4: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  categorical_cols = core_df.select_dtypes(include=["object"]).columns


In [4]:
for col in categorical_cols:
    print(col, core_df[col].nunique())


ProductCD 5
card4 4
card6 4
P_emaildomain 59
R_emaildomain 60
M1 2
M2 2
M3 2
M4 3
M5 2
M6 2
M7 2
M8 2
M9 2


In [5]:
categorical_tx_cols = list(categorical_cols)

categorical_tx_cols


['ProductCD',
 'card4',
 'card6',
 'P_emaildomain',
 'R_emaildomain',
 'M1',
 'M2',
 'M3',
 'M4',
 'M5',
 'M6',
 'M7',
 'M8',
 'M9']

In [6]:
encoded_cats = pd.get_dummies(
    train[categorical_tx_cols],
    drop_first=False,
    dummy_na=True
)

encoded_cats.shape


(590540, 165)

In [7]:
encoded_cats.head()


,ProductCD_C,ProductCD_H,ProductCD_R,ProductCD_S,ProductCD_W,ProductCD_nan,card4_american express,card4_discover,card4_mastercard,card4_visa,...,M6_nan,M7_F,M7_T,M7_nan,M8_F,M8_T,M8_nan,M9_F,M9_T,M9_nan
0,False,False,False,False,True,False,False,True,False,False,...,False,False,False,True,False,False,True,False,False,True
1,False,False,False,False,True,False,False,False,True,False,...,False,False,False,True,False,False,True,False,False,True
2,False,False,False,False,True,False,False,False,False,True,...,False,True,False,False,True,False,False,True,False,False
3,False,False,False,False,True,False,False,False,True,False,...,False,False,False,True,False,False,True,False,False,True
4,False,True,False,False,False,False,False,False,True,False,...,True,False,False,True,False,False,True,False,False,True


In [8]:
numeric_df = train[numeric_cols]

nunique = numeric_df.nunique()

constant_features = nunique[nunique <= 1]
near_constant_features = nunique[(nunique > 1) & (nunique <= 5)]

print("Constant numeric features:", len(constant_features))
print("Near-constant numeric features:", len(near_constant_features))


Constant numeric features: 0
Near-constant numeric features: 23


In [9]:
numeric_missing = numeric_df.isna().mean().sort_values(ascending=False)

numeric_missing.head(10)


dist2    0.936284
D7       0.934099
D13      0.895093
D14      0.894695
D12      0.890410
D6       0.876068
D8       0.873123
D9       0.873123
V148     0.861237
V138     0.861237
dtype: float64

In [10]:
numeric_df.describe().T[["min", "max", "mean", "std"]].head(10)


,min,max,mean,std
TransactionAmt,0.251,31937.391,135.027176,239.162522
card1,1000.000,18396.000,9898.734658,4901.170153
card2,100.000,600.000,362.555488,157.793246
card3,100.000,231.000,153.194925,11.336444
card5,100.000,237.000,199.278897,41.244453
addr1,100.000,540.000,290.733794,101.741072
addr2,10.000,102.000,86.800630,2.690623
dist1,0.000,10286.000,118.502180,371.872026
dist2,0.000,11623.000,231.855423,529.053494
C1,0.000,4685.000,14.092458,133.569018


In [11]:
numeric_tx = train[numeric_cols].copy()

numeric_tx.shape


(590540, 377)

In [12]:
X_baseline = pd.concat(
    [numeric_tx.reset_index(drop=True),
     encoded_cats.reset_index(drop=True)],
    axis=1
)

X_baseline.shape


(590540, 542)

In [13]:
y = train["isFraud"].copy()

X_baseline.shape, y.shape


((590540, 542), (590540,))

In [14]:
X_baseline.isna().mean().sort_values(ascending=False).head(10)


dist2    0.936284
D7       0.934099
D13      0.895093
D14      0.894695
D12      0.890410
D6       0.876068
D8       0.873123
D9       0.873123
V140     0.861237
V163     0.861237
dtype: float64

In [15]:
# Use median TransactionDT as split point
split_time = train["TransactionDT"].median()

train_idx = train["TransactionDT"] <= split_time
val_idx = train["TransactionDT"] > split_time

X_train = X_baseline.loc[train_idx]
X_val = X_baseline.loc[val_idx]

y_train = y.loc[train_idx]
y_val = y.loc[val_idx]

X_train.shape, X_val.shape


((295270, 542), (295270, 542))

In [17]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = X_train.copy()
X_val_scaled = X_val.copy()

X_train_scaled[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_val_scaled[numeric_cols] = scaler.transform(X_val[numeric_cols])


In [18]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(
    max_iter=2000,
    solver="lbfgs",
    n_jobs=-1
)

log_reg.fit(X_train_scaled, y_train)


c:\Projects\fraud-detection-ml\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [19]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression


In [20]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

# Pipeline for numeric columns: impute -> scale
numeric_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

# For one-hot categorical columns: pass through (they’re already 0/1 + dummy_na)
preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_pipe, numeric_cols),
        ("cat", "passthrough", list(encoded_cats.columns))
    ],
    remainder="drop",
    sparse_threshold=0.3
)

log_reg = LogisticRegression(
    max_iter=2000,
    solver="lbfgs",
    n_jobs=-1
)

clf = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", log_reg)
])


In [21]:
clf.fit(pd.concat([train[numeric_cols], encoded_cats], axis=1).loc[train_idx], y_train)


c:\Projects\fraud-detection-ml\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocess', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers cont

In [22]:
X_full_baseline = pd.concat(
    [train[numeric_cols].reset_index(drop=True),
     encoded_cats.reset_index(drop=True)],
    axis=1
)

X_train_full = X_full_baseline.loc[train_idx]
X_val_full = X_full_baseline.loc[val_idx]


In [23]:
clf.fit(X_train_full, y_train)


c:\Projects\fraud-detection-ml\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocess', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers cont

In [24]:
from sklearn.metrics import precision_recall_curve, auc

y_val_probs = clf.predict_proba(X_val_full)[:, 1]

precision, recall, _ = precision_recall_curve(y_val, y_val_probs)
pr_auc = auc(recall, precision)

pr_auc


0.2659546844963069

In [25]:
from sklearn.metrics import precision_score, recall_score

thr = 0.5
y_val_pred = (y_val_probs >= thr).astype(int)

precision_at_thr = precision_score(y_val, y_val_pred, zero_division=0)
recall_at_thr = recall_score(y_val, y_val_pred)

precision_at_thr, recall_at_thr


(0.5472541507024266, 0.2387408301606463)

In [26]:
for thr in [0.5, 0.3, 0.2, 0.1, 0.05]:
    y_pred = (y_val_probs >= thr).astype(int)
    p = precision_score(y_val, y_pred, zero_division=0)
    r = recall_score(y_val, y_pred)
    print(f"threshold={thr:>4} | precision={p:.3f} | recall={r:.3f}")


threshold= 0.5 | precision=0.547 | recall=0.239
threshold= 0.3 | precision=0.479 | recall=0.295
threshold= 0.2 | precision=0.403 | recall=0.349
threshold= 0.1 | precision=0.281 | recall=0.464
threshold=0.05 | precision=0.173 | recall=0.595


## Baseline Model Evaluation — Logistic Regression

### Validation Strategy
A time-based split was used to reflect real-world deployment, training on earlier transactions and validating on later ones.  
Random splitting was avoided to prevent temporal leakage, as fraud prevalence and behavior vary over time.

### Overall Performance
The baseline Logistic Regression model achieved a PR-AUC of approximately **0.27**, indicating meaningful separation between fraudulent and legitimate transactions given the class imbalance.

This confirms that the engineered baseline features contain real predictive signal without relying on identity or time-derived features.

### Threshold Trade-off Analysis
Model probabilities were converted into binary decisions using different thresholds to illustrate the precision–recall trade-off:

- **High threshold (0.5)**: High precision (~55%) but low recall (~24%), catching only the most obvious fraud.
- **Moderate thresholds (0.2–0.3)**: More balanced trade-off between fraud capture and false positives.
- **Low threshold (0.05–0.1)**: High recall (~46–60%) at the cost of lower precision, suitable for low-friction interventions.

This demonstrates that model behavior can be tuned via threshold selection without retraining, depending on operational constraints.

### Key Takeaways
- Baseline transaction-level features are sufficient to learn non-trivial fraud patterns.
- Threshold selection plays a critical role in operational performance.
- Accuracy is not an appropriate metric for this problem due to extreme class imbalance.


---

### End of Baseline Modeling Phase

At this stage, a complete baseline fraud detection pipeline has been established using
transaction-level features only, including preprocessing, time-based validation,
and threshold-based evaluation.

Subsequent work will focus on model interpretation, comparison with non-linear models,
and feature engineering enhancements.


## Model Interpretation — Logistic Regression Coefficients

To understand which baseline features drive fraud predictions, we inspect the learned
coefficients of the Logistic Regression model. Since numeric features were standardized,
coefficient magnitudes are directly comparable across features.

Positive coefficients indicate increased fraud risk, while negative coefficients
indicate lower fraud risk.


In [27]:
# Get feature names after preprocessing
num_features = numeric_cols
cat_features = encoded_cats.columns

feature_names = list(num_features) + list(cat_features)

len(feature_names)


542

In [28]:
import pandas as pd

coefs = clf.named_steps["model"].coef_[0]

coef_df = pd.DataFrame({
    "feature": feature_names,
    "coefficient": coefs
}).sort_values("coefficient", ascending=False)

coef_df.head(10), coef_df.tail(10)


(                          feature  coefficient
 19                            C11     3.586275
 16                             C8     2.307784
 12                             C4     2.248880
 422        P_emaildomain_mail.com     1.865708
 377                   ProductCD_C     1.761139
 431  P_emaildomain_protonmail.com     1.481628
 482        R_emaildomain_mail.com     1.349821
 513             R_emaildomain_nan     1.275123
 476      R_emaildomain_icloud.com     1.263038
 18                            C10     1.180333,
                          feature  coefficient
 9                             C1    -1.183339
 205                         V168    -1.233892
 436  P_emaildomain_sbcglobal.net    -1.335305
 529                       M5_nan    -1.392889
 304                         V267    -1.430158
 478       R_emaildomain_live.com    -1.578584
 401    P_emaildomain_charter.net    -2.061951
 381                  ProductCD_W    -2.130159
 15                            C7    -2.289856
 

### Observations from Coefficient Analysis

The strongest positive coefficients correspond to transaction patterns that the model
associates with elevated fraud risk, while strong negative coefficients indicate patterns
more commonly observed in legitimate transactions.

Several high-magnitude coefficients originate from engineered numeric transaction features,
suggesting that even without identity or temporal context, meaningful behavioral signals
are present.

As expected, categorical indicators such as product type and transaction flags also
contribute directionally to fraud risk.


### Interpretation Summary

The baseline Logistic Regression model relies on a combination of anonymized numeric
transaction features and low-cardinality categorical indicators to assess fraud risk.

Certain email domains and the absence of recipient email information are associated with
higher predicted fraud risk, while specific product categories and other domains act as
legitimacy signals. These patterns align with common fraud behaviors and validate earlier
EDA findings regarding the importance of missingness as an informative signal.

No evidence of identifier leakage or temporal leakage was observed in the learned
coefficients, supporting the validity of the baseline pipeline.


### Interpretation Summary

The baseline Logistic Regression model relies on a combination of anonymized numeric
transaction features and low-cardinality categorical indicators to assess fraud risk.

Certain email domains and the absence of recipient email information are associated with
higher predicted fraud risk, while specific product categories and other domains act as
legitimacy signals. These patterns align with common fraud behaviors and validate earlier
EDA findings regarding the importance of missingness as an informative signal.

No evidence of identifier leakage or temporal leakage was observed in the learned
coefficients, supporting the validity of the baseline pipeline.


In [29]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(
    max_depth=6,
    min_samples_leaf=1000,
    random_state=42
)

tree.fit(X_train_full, y_train)


,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.",'gini'
,"splitter splitter: {""best"", ""random""}, default=""best""The strategy used to choose the split at each node. Supportedstrategies are ""best"" to choose the best split and ""random"" to choosethe best random split.",'best'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",6
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1000
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: int, float or {""sqrt"", ""log2""}, default=NoneThe number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... note:: The search for a split does not stop until at least one valid partition of the node samples is found, even if it requires to effectively inspect more than ``max_features`` features.",None
,"random_state random_state: int, RandomState instance or None, default=NoneControls the randomness of the estimator. The features are alwaysrandomly permuted at each split, even if ``splitter`` is set to``""best""``. When ``max_features < n_features``, the algorithm willselect ``max_features`` at random at each split before finding the bestsplit among them. But the best found split may vary across differentruns, even if ``max_features=n_features``. That is the case, if theimprovement of the criterion is identical for several splits and onesplit has to be selected at random. To obtain a deterministic behaviourduring fitting, ``random_state`` has to be fixed to an integer.See :term:`Glossary ` for details.",42
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow a tree with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current

In [30]:
from sklearn.metrics import precision_recall_curve, auc

y_val_probs_tree = tree.predict_proba(X_val_full)[:, 1]

precision_t, recall_t, _ = precision_recall_curve(y_val, y_val_probs_tree)
pr_auc_tree = auc(recall_t, precision_t)

pr_auc_tree


0.35423272351857815

In [31]:
for thr in [0.5, 0.3, 0.2, 0.1, 0.05]:
    y_pred = (y_val_probs_tree >= thr).astype(int)
    p = precision_score(y_val, y_pred, zero_division=0)
    r = recall_score(y_val, y_pred)
    print(f"threshold={thr:>4} | precision={p:.3f} | recall={r:.3f}")


threshold= 0.5 | precision=0.841 | recall=0.130
threshold= 0.3 | precision=0.521 | recall=0.298
threshold= 0.2 | precision=0.370 | recall=0.364
threshold= 0.1 | precision=0.320 | recall=0.414
threshold=0.05 | precision=0.179 | recall=0.572


### Decision Tree vs Logistic Regression — Summary

The shallow Decision Tree achieved a higher PR-AUC than the Logistic Regression baseline,
indicating that non-linear interactions between transaction-level features contribute
meaningfully to fraud detection.

At higher thresholds, the tree exhibits very high precision but low recall, reflecting
its tendency to learn sharp, high-confidence rules. In contrast, Logistic Regression
provides smoother risk ranking with broader coverage.

These results suggest that more expressive models can outperform linear baselines,
particularly when feature interactions are present, and motivate the introduction of
additional behavioral features.
